In [ ]:
!nvidia-smi
import os
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/My Drive'
os.chdir(path)
os.listdir(path)

In [ ]:
#input channel RedGreenBlue RGB 同一个像素点对不同颜色的敏感程度，取值多为(0~255)
#图片左上角是原点(0,0)，width为图片横长，height为纵宽

import torch
in_channel, out_channel = 5,10
width,height = 100,100
kernel_size = 3
batch_size = 1

input = torch.randn(batch_size,
            in_channel,
            width,
            height)
conv_layer = torch.nn.Conv2d(in_channel,  #输入channel的大小
                out_channel, #输出channel的大小
                padding =1, #在输入的图形外部加1圈数值为0的padding
                stride =2,  #步长 
                kernel_size = kernel_size)  #卷积核大小

output1 = conv_layer(input)

print(input.shape)
print(output1.shape)
print(conv_layer.weight.shape)
#print(conv_layer.weight.data)


#MaxPooling
maxpooling_layer = torch.nn.MaxPool2d(kernel_size=kernel_size)#一个3*3的kernel,取其中最大的值作为结果。默认stride步也被设置为kernel_size,
output2=maxpooling_layer(input)
print(output2.shape)

torch.Size([1, 5, 100, 100])
torch.Size([1, 10, 50, 50])
torch.Size([10, 5, 3, 3])
torch.Size([1, 5, 33, 33])


# MNIST

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets
import torch.nn.functional as F


class Model(torch.nn.Module):
  def __init__(self):
    super(Model,self).__init__()
    self.conv1 = torch.nn.Conv2d(1,10,kernel_size=5)
    self.conv2 = torch.nn.Conv2d(10,20,kernel_size=5)
    self.pooling = torch.nn.MaxPool2d(kernel_size=2)
    self.fc = torch.nn.Linear(320,10)
  
  def forward(self, x):
    batch_size = x.size(0)
    x = F.relu(self.pooling(self.conv1(x)))#与图不同，先MaxPool再ReLU
    x = F.relu(self.pooling(self.conv2(x)))
    x = x.view(batch_size,-1)#将一个图再次合并到一起，batch_size个图组成一个batch
    x = self.fc(x)
    return x 

batch_size = 64
transform = transforms.Compose([
  transforms.ToTensor(),          
  transforms.Normalize((0.1307,),(0.3081,))                
])

train_set = datasets.MNIST(root='Colab Notebooks/dataset',
              train=True,           
              transform=transform,       
              download=True)
test_set = datasets.MNIST(root='Colab Notebooks/dataset',
              train=False,          
              transform=transform,
              download=True)

#加载数据集
train_loader = DataLoader(dataset=train_set,
              batch_size=batch_size,
              shuffle=True)
test_loader = DataLoader(dataset=test_set,
              batch_size=batch_size,
              shuffle=False)

model = Model()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

#使用GPU计算
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)

def train(epoch):
  running_loss = 0.0
  for batch_idx,(input,target) in enumerate(train_loader):
    input,target = input.to(device), target.to(device)#使用GPU计算
    optimizer.zero_grad()
    output = model(input)
    loss = criterion(output,target)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if batch_idx % 300 == 299:
      print('[%d,%5d] loss: %.3f' % (epoch + 1, batch_idx +1, running_loss/300))
      running_loss = 0.0

def test():
  correct = 0
  total = 0
  with torch.no_grad(): 
    for input,target in test_loader:
      input,target = input.to(device), target.to(device)#使用GPU计算
      output = model(input)
      _,predicted = torch.max(output.data,dim=1)
      total+=target.size(0)
      correct += (predicted == target).sum().item()
  print('Accuracy on test set:%d %%' % (100*correct/total))


for epoch in range(10):
  train(epoch)
  test()

[1,  300] loss: 0.648
[1,  600] loss: 0.196
[1,  900] loss: 0.148
Accuracy on test set:96 %
[2,  300] loss: 0.115
[2,  600] loss: 0.106
[2,  900] loss: 0.091
Accuracy on test set:97 %
[3,  300] loss: 0.086
[3,  600] loss: 0.072
[3,  900] loss: 0.070
Accuracy on test set:98 %
[4,  300] loss: 0.068
[4,  600] loss: 0.062
[4,  900] loss: 0.059
Accuracy on test set:98 %
[5,  300] loss: 0.052
[5,  600] loss: 0.054
[5,  900] loss: 0.058
Accuracy on test set:98 %
[6,  300] loss: 0.049
[6,  600] loss: 0.046
[6,  900] loss: 0.049
Accuracy on test set:98 %
[7,  300] loss: 0.044
[7,  600] loss: 0.041
[7,  900] loss: 0.047
Accuracy on test set:98 %
[8,  300] loss: 0.037
[8,  600] loss: 0.039
[8,  900] loss: 0.046
Accuracy on test set:98 %
[9,  300] loss: 0.034
[9,  600] loss: 0.036
[9,  900] loss: 0.042
Accuracy on test set:98 %
[10,  300] loss: 0.034
[10,  600] loss: 0.036
[10,  900] loss: 0.035
Accuracy on test set:98 %
